In [ ]:
!pip install openai google huggingface_hub google-genai gradio transformers torch bitsandbytes requests accelerate sentencepiece -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.5 MB/s eta 0:00:00


In [ ]:
import os
import io
import sys
from openai import OpenAI
from google import genai
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
import platform
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig, TextStreamer
import torch

In [ ]:
openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

google_key = userdata.get('GOOGLE_API_KEY')

In [ ]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time. Keep implementations of random number generators identical so that results match exactly."

In [ ]:
def user_prompt(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [ ]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt(python)}
    ]

In [ ]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
hf_token = userdata.get('HF_TOKEN')

In [ ]:
NxCode = 'NTQAI/Nxcode-CQ-7B-orpo'

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(NxCode)
tokenizer.pad_token = tokenizer.eos_token
messages = messages_for(pi)

model = AutoModelForCausalLM.from_pretrained(
    NxCode,
    device_map="auto",
    quantization_config=quant_config
)

tokenizer_config.json:   0%|          | 0.00/972 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.46M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/6.55G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/7.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

In [ ]:
import re

def generate_code_NxCode(model, tokenizer, messages):
    inputs = tokenizer.apply_chat_template(messages, return_tensors='pt',add_generation_prompt=False).to('cuda')
    streamer = TextStreamer(tokenizer, skip_prompt=True)
    outputs = model.generate(inputs, max_new_tokens=2000)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract C++ code block between ```cpp ... ```
    match = re.search(r"```cpp(.*?)```", response, re.DOTALL)
    if match:
        cpp_code = match.group(1).strip()
    else:
        cpp_code = "C++ code not found."

    return cpp_code


In [ ]:
cpp_code_NxCode = generate_code_NxCode(model, tokenizer, messages)
print(cpp_code_NxCode)

C++ code not found.


In [ ]:
def extract_cpp_code(python):
    messages = messages_for(python)
    inputs = tokenizer.apply_chat_template(messages, return_tensors='pt', add_generation_prompt=False).to('cuda')
    outputs = model.generate(inputs, max_new_tokens=2000)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    match = re.search(r"```cpp(.*?)```", response, re.DOTALL)
    return match.group(1).strip() if match else "C++ code not found."

In [ ]:
extract_cpp_code(pi)

'#include <iostream>\n#include <iomanip>\n#include <chrono>\nusing namespace std;\n\ndouble calculate(int iterations, int param1, int param2) {\n    double result = 1.0;\n    for (int i = 1; i <= iterations; ++i) {\n        double j = static_cast<double>(i * param1) - param2;\n        result -= 1.0 / j;\n        j = static_cast<double>(i * param1) + param2;\n        result += 1.0 / j;\n    }\n    return result;\n}\n\nint main() {\n    chrono::time_point<chrono::high_resolution_clock> start_time = chrono::high_resolution_clock::now();\n    double result = calculate(100000000, 4, 1) * 4;\n    chrono::time_point<chrono::high_resolution_clock> end_time = chrono::high_resolution_clock::now();\n    chrono::duration<double, milli> elapsed = end_time - start_time;\n\n    cout << fixed << setprecision(12) << "Result: " << result << endl;\n    cout << "Execution Time: " << elapsed.count() << " milliseconds" << endl;\n\n    return 0;\n}'

In [ ]:
with gr.Blocks() as ui :
  gr.Markdown('# Convert Python to C++')

  with gr.Row():
      python = gr.Textbox(label="Python code:", value=pi, lines=10)
      cpp = gr.Textbox(label="C++ code:", lines=10)

  with gr.Row():
      convert = gr.Button("Convert code")

  convert.click(extract_cpp_code, inputs=[python], outputs=[cpp])

ui.launch(share=True,debug=True)